In [1]:

import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Embedding, LSTM, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from PIL import Image


In [2]:
# --- SETTINGS ---
IMAGE_SIZE = (224, 224)
MAX_LEN = 100
VOCAB_SIZE = 5000
BATCH_SIZE = 32

In [3]:



# CSV_FILE = "symptoms.csv"  # path to your CSV file
# IMAGE_FOLDER = r"C:\Users\User\Downloads\Skin-disease-dataset\train"   # root folder containing subfolders of images

# # Load CSV
# df = pd.read_csv(CSV_FILE)
# df.dropna(inplace=True)

# # Append full image path
# df["image_path"] = df.apply(lambda row: os.path.join(IMAGE_FOLDER, row["label"], row["image_name"]), axis=1)

# # Encode labels
# label_encoder = LabelEncoder()
# df["label_id"] = label_encoder.fit_transform(df["label"])

# # Tokenize text
# tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
# tokenizer.fit_on_texts(df["symptom"])


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer

# Settings
CSV_FILE = "symptoms.csv"
IMAGE_FOLDER = r"C:\Users\User\Downloads\Skin-disease-dataset\train"   # dataset/acne/, dataset/psoriasis/, etc.

# Load CSV
df = pd.read_csv(CSV_FILE)
df.dropna(inplace=True)

# Label encode
label_encoder = LabelEncoder()
df["label_id"] = label_encoder.fit_transform(df["label"])

# Tokenize text
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["symptom"])

# Bind an image path from the correct folder
image_paths = []
for label in df["label"]:
    label_dir = os.path.join(IMAGE_FOLDER, label)
    if not os.path.isdir(label_dir):
        raise ValueError(f"Folder not found for label: {label}")
    
    image_list = [f for f in os.listdir(label_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if not image_list:
        raise ValueError(f"No images found in folder: {label_dir}")
    
    # Pick a random image OR use index % len(images) if you want balance
    img_path = os.path.join(label_dir, np.random.choice(image_list))
    image_paths.append(img_path)

# Add image paths to dataframe
df["image_path"] = image_paths


In [ ]:
# Save the updated DataFrame
output_csv = "multimodal_dataset.csv"
df.to_csv(output_csv, index=False)
print(f"✅ Progress saved to: {output_csv}")

✅ Progress saved to: multimodal_dataset.csv


In [ ]:
df = pd.read_csv("multimodal_dataset.csv")

In [ ]:

def preprocess(image_path, token_ids, label):
    try:
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [224, 224])
        image = image / 255.0

        return image, token_ids, tf.cast(label, tf.int64)

    except Exception as e:
        print(f"❌ Error: {image_path.numpy().decode()}")
        return tf.zeros([224, 224, 3]), tf.zeros([MAX_LEN], dtype=tf.int32), tf.constant(-1, dtype=tf.int64)

def tf_preprocess(image_path, token_ids, label):
    image, tokens, label = tf.py_function(preprocess, [image_path, token_ids, label], [tf.float32, tf.int32, tf.int64])
    image.set_shape([224, 224, 3])
    tokens.set_shape([MAX_LEN])
    label.set_shape([])
    return {"image_input": image, "text_input": tokens}, label


dataset = tf.data.Dataset.from_tensor_slices((df["image_path"], df["token_ids"], df["label_id"]))
dataset = dataset.map(tf_preprocess)
dataset = dataset.filter(lambda data, label: tf.not_equal(label, -1))
dataset = dataset.shuffle(len(df)).batch(BATCH_SIZE)


: 

In [ ]:
for data, label in dataset.take(1):
    print("✅ Sample loaded")
    print("Image shape:", data["image_input"].shape)
    print("Text shape:", data["text_input"].shape)
    print("Label:", label.numpy())


❌ Error processing: C:\Users\User\Downloads\Skin-disease-dataset\train\Acne\pigmentation_0_1889.jpeg, Small red, tender bumps called papules that can get worse.
   ⚠️ Exception: Failed copying input tensor from /job:localhost/replica:0/task:0/device:GPU:0 to /job:localhost/replica:0/task:0/device:CPU:0 in order to run ResizeBilinear: Dst tensor is not initialized. [Op:ResizeBilinear]
❌ Error processing: C:\Users\User\Downloads\Skin-disease-dataset\train\Acne\aug_3761_acne-histology-5.jpg, Small red, tender bumps called papules
   ⚠️ Exception: {{function_node __wrapped__DecodeJpeg_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[488,720,3] and type uint8 on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:DecodeJpeg]
❌ Error processing: C:\Users\User\Downloads\Skin-disease-dataset\train\Acne\aug_128_hidradenitis-suppurativa-64.jpg, Small red, tender bumps called papules mostly visible.
   ⚠️ Exception: Failed copying input ten

In [ ]:
def filter_invalid(data, label):
    return tf.not_equal(label, -1)

dataset = dataset.filter(filter_invalid)

ValueError: Invalid `predicate`. `predicate` must return a `tf.bool` scalar tensor, but its return type is TensorSpec(shape=(None,), dtype=tf.bool, name=None).

In [ ]:

# Image branch
img_input = Input(shape=(224, 224, 3), name="image_input")
cnn_base = MobileNetV2(include_top=False, weights='imagenet', input_tensor=img_input)
cnn_base.trainable = False
x1 = GlobalAveragePooling2D()(cnn_base.output)

# Text branch
txt_input = Input(shape=(MAX_LEN,), name="text_input")
x2 = Embedding(input_dim=VOCAB_SIZE, output_dim=64, mask_zero=True)(txt_input)
x2 = LSTM(64)(x2)

# Combine
merged = Concatenate()([x1, x2])
x = Dropout(0.3)(merged)
x = Dense(64, activation='relu')(x)
output = Dense(len(label_encoder.classes_), activation='softmax')(x)

model = Model(inputs=[img_input, txt_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['image_input[0][0]']            
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [ ]:

# Train the model
model.fit(dataset, epochs=10)


Epoch 1/10


InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  ValueError: Tensor conversion requested dtype int64 for Tensor with dtype int32: <tf.Tensor: shape=(), dtype=int32, numpy=0>
Traceback (most recent call last):

  File "c:\Users\User\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\ops\script_ops.py", line 269, in __call__
    return func(device, token, args)

  File "c:\Users\User\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\ops\script_ops.py", line 147, in __call__
    outputs = self._call(device, args)

  File "c:\Users\User\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\ops\script_ops.py", line 163, in _call
    outputs = [

  File "c:\Users\User\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\ops\script_ops.py", line 164, in <listcomp>
    _maybe_copy_to_context_device(self._convert(x, dtype=dtype),

  File "c:\Users\User\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\ops\script_ops.py

In [ ]:

def load_image(img_path):
    img = Image.open(img_path).convert("RGB")
    img = img.resize(IMAGE_SIZE)
    img = np.array(img) / 255.0
    return img

def predict_disease(image=None, text=None):
    if image is None:
        image = np.zeros((224, 224, 3))
    else:
        image = load_image(image)
    image = np.expand_dims(image, axis=0)

    if text is None:
        text = "<no description>"
    tokens = tokenizer.texts_to_sequences([text])
    tokens = pad_sequences(tokens, maxlen=MAX_LEN)

    preds = model.predict({"image_input": image, "text_input": tokens})
    pred_idx = np.argmax(preds)
    return label_encoder.classes_[pred_idx]

# Example usage:
# predict_disease(image="dataset/acne/img1.jpg", text="red bumps on face")
